In [1]:
import sys
sys.path.append('../Util')
import pickle
import os

In [2]:
from evaluation import correlation, lr, perceptron, get_anova_dims, get_mi_dims
from preparation import prepare_dataset, read_datasets

In [3]:
from we import get_we, initiate_model

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from scipy.stats import f_oneway

In [6]:
from sklearn.metrics import mean_absolute_error, accuracy_score

In [7]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
models = [
    {
        'name': 'flaubert/flaubert_small_cased',
        'label': 'flau_small_c'
    },
    {
    
        'name': 'flaubert/flaubert_base_uncased', 
        'label': 'flau_base_u'

    },
    {
        'name': 'flaubert/flaubert_base_cased',
        'label': 'flau_base_c'
    },
    {
        'name': 'flaubert/flaubert_large_cased',
        'label': 'flau_large_c'
    },
    {
        'name': 'camembert/camembert-base',
        'label': 'cam_base'
    },
    {
        'name': 'xlm-roberta-large',
        'label': 'xlm_large'
    },
    {
    
        'name': 'xlm-roberta-base', 
        'label': 'xlm_base'

    },
    {
        'name': 'bert-base-multilingual-uncased',
        'label': 'bert_base_u'
    },
    {
        'name': 'distilbert-base-multilingual-cased',
        'label': 'distilbert_base'
    },
    {
        'name': 'bert-base-multilingual-cased',
        'label': 'bert_base_c'
    }
    
]

In [9]:
labels = [m['label'] for m in models]

In [10]:
we_with_features = read_datasets(
                            path = '../Data',
                            model_labels = labels,
                            file_name = 'all_unique_pos_we.csv'
                    )

In [11]:
we_with_features[-1].head()

,0,1,2,3,4,5,6,7,8,9,...,764,765,766,767,Number,Gender,Lemma,POS,Tense,Person
Word,,,,,,,,,,,,,,,,,,,,,
2D,0.909603,0.551025,0.436937,0.789782,0.311517,0.417476,0.629341,0.913722,0.349834,0.838988,...,0.285154,0.057887,0.030874,0.310204,invariable,feminine,2D,NOUN,NaN,NaN
3D,0.901779,0.549880,0.427826,0.797102,0.313900,0.419226,0.623478,0.916539,0.355807,0.844714,...,0.287610,0.054251,0.018604,0.305226,invariable,feminine,3D,NOUN,NaN,NaN
aa,0.899570,0.555891,0.418186,0.805754,0.319232,0.412976,0.618457,0.914183,0.367882,0.845985,...,0.287883,0.057476,0.013614,0.290463,invariable,masculine,aa,NOUN,NaN,NaN
abandon,0.898879,0.556306,0.417821,0.805256,0.318463,0.412505,0.618276,0.914983,0.367654,0.846577,...,0.288183,0.057827,0.014050,0.290994,singular,masculine,abandon,NOUN,NaN,NaN
abbaye,0.895888,0.554461,0.420659,0.799339,0.322441,0.418141,0.618820,0.921033,0.367005,0.845904,...,0.293985,0.057312,0.012175,0.298473,singular,feminine,abbaye,NOUN,NaN,NaN


In [12]:
feature_col_count = 6

# Feature to investigate in this notebook
feature = 'POS'

# Nouns

In [13]:
pos = ['NOUN']

Split each model into train and test:

In [14]:
X_noun_train = []
y_noun_train = []

X_noun_test = []
y_noun_test = []

In [15]:
for we in we_with_features:
    xtr, xtst, ytr, ytst = prepare_dataset(dataset=we[(we.Gender != 'invariable') & (we.Number != 'invariable')],
                                           feature_col_count=feature_col_count,
                                           feature_name=feature,
                                           normalize=False,
                                           encode=True,
                                           encode_as1='NOUN',
                                           split=True,
                                           balance=True)
    X_noun_train.append(xtr)
    X_noun_test.append(xtst)
    
    y_noun_train.append(ytr)
    y_noun_test.append(ytst)

In [16]:
dataset_sizes = pd.DataFrame(index=labels)
dataset_sizes['Train size'] = [len(x) for x in X_noun_train]
dataset_sizes['Test size'] = [len(x) for x in X_noun_test]
dataset_sizes

,Train size,Test size
flau_small_c,8110,2028
flau_base_u,9656,2414
flau_base_c,8110,2028
flau_large_c,8110,2028
cam_base,4784,1196
xlm_large,1713,429
xlm_base,1713,429
bert_base_u,2628,658
distilbert_base,1347,337
bert_base_c,1347,337


## Non-independent dims

ANOVA test with p-value < 0.01:

In [17]:
anova_dims = [get_anova_dims(X_noun_train[i], y_noun_train[i]) for i in range(len(models))]

In [18]:
mi_dims = [get_mi_dims(X_noun_train[i], y_noun_train[i]) for i in range(len(models))]

In [19]:
non_ind_dims = [list(set(anova_dims[i]).intersection(mi_dims[i])) for i in range(len(models))]

Stats about the number of dimensions for each model:

In [20]:
non_ind_df = pd.DataFrame(index=labels, columns=['ANOVA', 'MI', 'Total non independent'])

In [21]:
non_ind_df['ANOVA'] = [len(x) for x in anova_dims]
non_ind_df['MI'] = [len(x) for x in mi_dims]
non_ind_df['Total non independent'] = [len(x) for x in non_ind_dims]

In [22]:
non_ind_df

,ANOVA,MI,Total non independent
flau_small_c,442,435,396
flau_base_u,581,768,581
flau_base_c,557,576,442
flau_large_c,851,858,753
cam_base,234,500,156
xlm_large,494,655,360
xlm_base,79,483,55
bert_base_u,405,358,196
distilbert_base,332,475,236
bert_base_c,6,585,6


In [23]:
dims = {}

In [24]:
for i in range(len(models)):
    dims[labels[i]] = {}
    dims[labels[i]]['All dims'] = X_noun_train[i].columns
    dims[labels[i]]['ANOVA'] = anova_dims[i]
    dims[labels[i]]['MI'] = mi_dims[i]
    dims[labels[i]]['All non ind'] = non_ind_dims[i]

## Important dims

We can test different $\alpha$ values: 1%, 5%, 10%, 25%, 50%, 75%.

In [25]:
alphas = [1, 5, 10, 25, 50, 75]

Train Logistic Regression on train set for each model:

In [26]:
lr_res = [lr(X_noun_train[i], y_noun_train[i]) for i in range(len(models))]

Train Perceptron 10 times and get average weights:

In [27]:
perceptron_res = [perceptron(X_noun_train[i], y_noun_train[i]) for i in range(len(models))]

Compute correlation to the gender vector:

In [28]:
corr_res = [correlation(X_noun_train[i], y_noun_train[i]) for i in range(len(models))]

In [29]:
for i in range(len(models)):
    for alpha in alphas:
        num_imp_dims = len(X_noun_test[i].columns)*alpha//100
        lr_dims = [str(x[0]) for x in lr_res[i][:num_imp_dims]]
        perc_dims = [str(x[0]) for x in perceptron_res[i][:num_imp_dims]]
        corr_dims = [str(x[0]) for x in corr_res[i][:num_imp_dims]]
        dims[labels[i]][f'LR{alpha}'] = lr_dims
        dims[labels[i]][f'Perc{alpha}'] = perc_dims
        dims[labels[i]][f'Corr{alpha}'] = corr_dims
        dims[labels[i]][f'All imp dims{alpha}'] = list(set(lr_dims).intersection(perc_dims).intersection(corr_dims))

## Compute medians

In [30]:
medians = {}

In [31]:
for i in range(len(models)):
    medians[labels[i]] = {}
    for dim_group in dims[labels[i]].keys():
        medians[labels[i]][dim_group] = {}
        dim_list = dims[labels[i]][dim_group]
        # Median of dimensions where feature vector is equal to 0
        median_0 = X_noun_train[i][y_noun_train[i] == 0][dim_list].median()
        # Median of dimensions where feature vector is equal to 1
        median_1 = X_noun_train[i][y_noun_train[i] == 1][dim_list].median()
        
        medians[labels[i]][dim_group]['0'] = median_0
        medians[labels[i]][dim_group]['1'] = median_1


In [32]:
dim_lens = {}

for model in dims.keys():
    dim_lens[model] = {}
    for dim_group in dims[model].keys():
        dim_lens[model][dim_group] = len(dims[model][dim_group])

In [33]:
pd.DataFrame(dim_lens)

,flau_small_c,flau_base_u,flau_base_c,flau_large_c,cam_base,xlm_large,xlm_base,bert_base_u,distilbert_base,bert_base_c
All dims,512,768,768,1024,768,1024,768,768,768,768
ANOVA,442,581,557,851,234,494,79,405,332,6
MI,435,768,576,858,500,655,483,358,475,585
All non ind,396,581,442,753,156,360,55,196,236,6
LR1,5,7,7,10,7,10,7,7,7,7
Perc1,5,7,7,10,7,10,7,7,7,7
Corr1,5,7,7,10,7,10,7,7,7,7
All imp dims1,0,1,0,1,1,0,2,1,0,0
LR5,25,38,38,51,38,51,38,38,38,38
Perc5,25,38,38,51,38,51,38,38,38,38


## Compute predictions

In [34]:
y_preds = {}

In [35]:
for i in range(len(models)):
    y_preds[labels[i]] = {}
    for dim_group in dims[labels[i]].keys():
        dim_list = dims[labels[i]][dim_group]
        mae0 = X_noun_test[i][dim_list].apply(lambda x: mean_absolute_error(medians[labels[i]][dim_group]['0'], x), axis=1)
        mae1 = X_noun_test[i][dim_list].apply(lambda x: mean_absolute_error(medians[labels[i]][dim_group]['1'], x), axis=1)
        # If MSE for sample 0 is lower than for sample 1, the label should be 0. So we need to convert False to 0.
        y_preds[labels[i]][dim_group] = (mae0 > mae1).apply(int)
    

## Compute accuracies

In [36]:
accs = {}

In [37]:
for i in range(len(models)):
    accs[labels[i]] = {}
    for dim_group in dims[labels[i]].keys():
        y_true = y_noun_test[i]
        y_pred = y_preds[labels[i]][dim_group]
        if any(y_pred):
            acc = accuracy_score(y_true, y_pred)
        else:
            acc = 0
        accs[labels[i]][dim_group] = acc

In [38]:
accs_df = pd.DataFrame(accs)
accs_df

,flau_small_c,flau_base_u,flau_base_c,flau_large_c,cam_base,xlm_large,xlm_base,bert_base_u,distilbert_base,bert_base_c
All dims,0.849606,0.588650,0.566568,0.873274,0.536789,0.547786,0.503497,0.544073,0.584570,0.534125
ANOVA,0.849606,0.601491,0.567061,0.873274,0.539298,0.545455,0.568765,0.542553,0.593472,0.543027
MI,0.849606,0.588650,0.567554,0.871795,0.536789,0.545455,0.505828,0.544073,0.578635,0.534125
All non ind,0.849112,0.601491,0.568047,0.872288,0.539298,0.547786,0.575758,0.544073,0.590504,0.543027
LR1,0.637081,0.601077,0.560651,0.722387,0.539298,0.468531,0.582751,0.544073,0.593472,0.534125
Perc1,0.791913,0.655344,0.605523,0.809665,0.543478,0.587413,0.545455,0.544073,0.560831,0.531157
Corr1,0.781558,0.657001,0.617357,0.816568,0.553512,0.550117,0.606061,0.545593,0.632047,0.537092
All imp dims1,0.000000,0.606877,0.000000,0.658284,0.505853,0.000000,0.596737,0.553191,0.000000,0.000000
LR5,0.797337,0.603977,0.566568,0.858481,0.540970,0.540793,0.573427,0.544073,0.596439,0.534125
Perc5,0.870316,0.690555,0.635108,0.874260,0.547659,0.545455,0.596737,0.545593,0.635015,0.540059


We can show what is the accuracy gain for each dimension test comparison to using all dimensions:

In [39]:
gains_df = (accs_df - accs_df.loc['All dims'])
gains_df

,flau_small_c,flau_base_u,flau_base_c,flau_large_c,cam_base,xlm_large,xlm_base,bert_base_u,distilbert_base,bert_base_c
All dims,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ANOVA,0.000000,0.012842,0.000493,0.000000,0.002508,-0.002331,0.065268,-0.001520,0.008902,0.008902
MI,0.000000,0.000000,0.000986,-0.001479,0.000000,-0.002331,0.002331,0.000000,-0.005935,0.000000
All non ind,-0.000493,0.012842,0.001479,-0.000986,0.002508,0.000000,0.072261,0.000000,0.005935,0.008902
LR1,-0.212525,0.012428,-0.005917,-0.150888,0.002508,-0.079254,0.079254,0.000000,0.008902,0.000000
Perc1,-0.057692,0.066694,0.038955,-0.063609,0.006689,0.039627,0.041958,0.000000,-0.023739,-0.002967
Corr1,-0.068047,0.068351,0.050789,-0.056706,0.016722,0.002331,0.102564,0.001520,0.047478,0.002967
All imp dims1,-0.849606,0.018227,-0.566568,-0.214990,-0.030936,-0.547786,0.093240,0.009119,-0.584570,-0.534125
LR5,-0.052268,0.015327,0.000000,-0.014793,0.004181,-0.006993,0.069930,0.000000,0.011869,0.000000
Perc5,0.020710,0.101906,0.068540,0.000986,0.010870,-0.002331,0.093240,0.001520,0.050445,0.005935


Here are the dimension groups and the accuracy gain this dimension group provides over all dimensions:

In [40]:
for i in range(len(models)):
    print(labels[i], gains_df.idxmax()[i], gains_df.loc[gains_df.idxmax()[i], labels[i]])

flau_small_c Perc25 0.03747534516765294
flau_base_u Perc5 0.10190555095277554
flau_base_c Perc5 0.06854043392504938
flau_large_c Perc25 0.038954635108481295
cam_base Corr1 0.01672240802675584
xlm_large Perc1 0.03962703962703962
xlm_base All imp dims5 0.11421911421911413
bert_base_u All imp dims1 0.00911854103343468
distilbert_base Perc5 0.05044510385756684
bert_base_c ANOVA 0.008902077151335286


In [41]:
for i in range(len(models)):
    print(labels[i], accs_df.idxmax()[i], accs_df.loc[accs_df.idxmax()[i], labels[i]])

flau_small_c Perc25 0.8870808678500987
flau_base_u Perc5 0.6905550952775477
flau_base_c Perc5 0.6351084812623274
flau_large_c Perc25 0.9122287968441815
cam_base Corr1 0.5535117056856187
xlm_large Perc1 0.5874125874125874
xlm_base All imp dims5 0.6177156177156177
bert_base_u All imp dims1 0.5531914893617021
distilbert_base Perc5 0.6350148367952523
bert_base_c ANOVA 0.543026706231454


In [42]:
if 'pos_noun.pickle' not in os.listdir('../Data/best_results/'):
    best_res = {}
    for label in labels:
        best_res[label] = {}
        best_res[label]['best_dim_set'] = []
        best_res[label]['best_dims'] = []
        best_res[label]['accs'] = []
        best_res[label]['gains'] = []
        best_res[label]['medians_0'] = []
        best_res[label]['medians_1'] = []


else:
    with open('../Data/best_results/pos_noun.pickle', 'rb') as f:
        best_res = pickle.load(f)

In [43]:
for i in range(len(models)):
    best_res[labels[i]]['best_dim_set'].append(gains_df.idxmax()[i])
    best_res[labels[i]]['best_dims'].append(dims[labels[i]][gains_df.idxmax()[i]])
    best_res[labels[i]]['accs'].append(accs_df.loc[accs_df.idxmax()[i], labels[i]])
    best_res[labels[i]]['gains'].append(gains_df.loc[gains_df.idxmax()[i], labels[i]])
    best_res[labels[i]]['medians_0'].append(medians[labels[i]][accs_df.idxmax()[i]]['0'])
    best_res[labels[i]]['medians_1'].append(medians[labels[i]][accs_df.idxmax()[i]]['1'])

In [44]:
with open('../Data/best_results/pos_noun.pickle', 'wb') as f:
    pickle.dump(best_res, f)

# Conclusion